In [ ]:
import torch
import numpy as np
from lieflow.groups import SO3
import matplotlib.pyplot as plt
from scipy.stats import special_ortho_group
from matplotlib.animation import FFMpegWriter
from mpl_toolkits.mplot3d import Axes3D
# %matplotlib widget

In [ ]:
so3 = SO3()

In [ ]:
R = torch.Tensor(special_ortho_group.rvs(3, size=1))
R, so3.log(R), (R - so3.exp(so3.log(R))).abs().max()

In [ ]:
R = torch.Tensor(special_ortho_group.rvs(3, size=50))
R = torch.Tensor(R)
(R - so3.exp(so3.log(R))).abs().max()

In [ ]:
R_0 = torch.eye(3)
R_1 = torch.Tensor(special_ortho_group.rvs(3, size=1))
A = so3.log(so3.L_inv(R_0, R_1))
ts = torch.linspace(0., 1., 100)
R_t = so3.L(R_0, so3.exp(ts[..., None, None] * A))

In [ ]:
skip = 10
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection="3d")
ax.quiver(ts[::skip], 0., 0., R_t[::skip, 0, 0], R_t[::skip, 1, 0], R_t[::skip, 2, 0], length=0.1, color="red")
ax.quiver(ts[::skip], 0., 0., R_t[::skip, 0, 1], R_t[::skip, 1, 1], R_t[::skip, 2, 1], length=0.1, color="green")
ax.quiver(ts[::skip], 0., 0., R_t[::skip, 0, 2], R_t[::skip, 1, 2], R_t[::skip, 2, 2], length=0.1, color="blue")
ax.quiver(ts[-1], 0., 0., R_t[-1, 0, 0], R_t[-1, 1, 0], R_t[-1, 2, 0], length=0.1, color="red")
ax.quiver(ts[-1], 0., 0., R_t[-1, 0, 1], R_t[-1, 1, 1], R_t[-1, 2, 1], length=0.1, color="green")
ax.quiver(ts[-1], 0., 0., R_t[-1, 0, 2], R_t[-1, 1, 2], R_t[-1, 2, 2], length=0.1, color="blue")
ax.set_xlim(0, 1)
ax.set_xlabel("x = t")
ax.set_ylim(-0.1, 0.1)
ax.set_yticks([])
ax.set_zlim(-0.1, 0.1)
ax.set_zticks([])
ax.set_aspect("equal");

In [ ]:
q_0 = torch.Tensor(((1., 0., 0.), (0., 0., 1.))).T
q_t = R_t @ q_0
q_1 = R_1 @ q_0

In [ ]:
theta = np.linspace(0, np.pi, 50)
phi = np.linspace(0, 2 * np.pi, 100)
theta, phi = np.meshgrid(theta, phi)

x = np.sin(theta) * np.cos(phi)
y = np.sin(theta) * np.sin(phi)
z = np.cos(theta)
r = 1.01

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')
ax.view_init(elev=15, azim=15)
ax.plot_surface(x, y, z, color='cyan', alpha=0.25, edgecolor=None)
ax.plot(r*q_t[:, 0, 0], r*q_t[:, 1, 0], r*q_t[:, 2, 0])
ax.quiver(
    r*q_t[10:-2:10, 0, 0], r*q_t[10:-2:10, 1, 0], r*q_t[10:-2:10, 2, 0],
    q_t[10:-2:10, 0, 1], q_t[10:-2:10, 1, 1], q_t[10:-2:10, 2, 1],
    length=0.25
)
ax.quiver(
    r*q_0[0, 0], r*q_0[1, 0], r*q_0[2, 0],
    q_0[0, 1], q_0[1, 1], q_0[2, 1],
    length=0.25, color="blue"
)
ax.quiver(
    r*q_1[0, 0], r*q_1[1, 0], r*q_1[2, 0],
    q_1[0, 1], q_1[1, 1], q_1[2, 1],
    length=0.25, color="green"
)

ax.set_xlim([-1, 1])
ax.set_xticks([])
ax.set_ylim([-1, 1])
ax.set_yticks([])
ax.set_zlim([-1, 1])
ax.set_zticks([])
ax.set_aspect("equal");


In [ ]:
metadata = {'title': 'Exponential Curve Interpolation SO(3)', 'artist': 'Matplotlib'}
writer = FFMpegWriter(fps=2, metadata=metadata)

R_0 = torch.eye(3)
fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot(111, projection="3d")
with writer.saving(fig, "output/exponential_curves_SO3.mp4", dpi=150):
    for frame in range(50):
        R_1 = torch.Tensor(special_ortho_group.rvs(3, size=1))

        A = so3.log(so3.L_inv(R_0, R_1))

        ts = torch.linspace(0., 1., 10)
        R_t = so3.L(R_0, so3.exp(ts[..., None, None] * A))
        
        ax.clear()
        ax.quiver(ts, 0., 0., R_t[:, 0, 0], R_t[:, 1, 0], R_t[:, 2, 0], length=0.1, color="red")
        ax.quiver(ts, 0., 0., R_t[:, 0, 1], R_t[:, 1, 1], R_t[:, 2, 1], length=0.1, color="green")
        ax.quiver(ts, 0., 0., R_t[:, 0, 2], R_t[:, 1, 2], R_t[:, 2, 2], length=0.1, color="blue")
        ax.set_xlim(0, 1)
        ax.set_xlabel("t")
        ax.set_ylim(-0.1, 0.1)
        ax.set_yticks([])
        ax.set_zlim(-0.1, 0.1)
        ax.set_zticks([])
        ax.set_aspect("equal")
        writer.grab_frame()

In [ ]:
metadata = {'title': 'Exponential Curve Interpolation W_2', 'artist': 'Matplotlib'}
writer = FFMpegWriter(fps=2, metadata=metadata)

R_0 = torch.eye(3)
q_0 = torch.Tensor(((1., 0., 0.), (0., 0., 1.))).T
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')
ax.view_init(elev=15, azim=15)
ax.plot_surface(x, y, z, color='cyan', alpha=0.25, edgecolor=None)
ax.quiver(r*q_0[0, 0], r*q_0[1, 0], r*q_0[2, 0], q_0[0, 1], q_0[1, 1], q_0[2, 1], length=0.25, color="blue")
quiver_inter = ax.quiver([], [], [], [], [], [])
quiver_end = ax.quiver([], [], [], [], [], [], color="green")
plot, = ax.plot([], [], [])

ax.set_xlim([-1, 1])
ax.set_xticks([])
ax.set_ylim([-1, 1])
ax.set_yticks([])
ax.set_zlim([-1, 1])
ax.set_zticks([])
ax.set_aspect("equal")
with writer.saving(fig, "output/exponential_curves_W2.mp4", dpi=150):
    for frame in range(50):
        R_1 = torch.Tensor(special_ortho_group.rvs(3, size=1))

        A = so3.log(so3.L_inv(R_0, R_1))

        ts = torch.linspace(0., 1., 100)
        R_t = so3.L(R_0, so3.exp(ts[..., None, None] * A))
        q_t = R_t @ q_0
        q_1 = R_1 @ q_0
        
        plot.remove()
        quiver_inter.remove()
        quiver_end.remove()

        plot, = ax.plot(r*q_t[:, 0, 0], r*q_t[:, 1, 0], r*q_t[:, 2, 0])
        quiver_inter = ax.quiver(
            r*q_t[10:-2:10, 0, 0], r*q_t[10:-2:10, 1, 0], r*q_t[10:-2:10, 2, 0],
            q_t[10:-2:10, 0, 1], q_t[10:-2:10, 1, 1], q_t[10:-2:10, 2, 1],
            length=0.25
        )
        quiver_end = ax.quiver(
            r*q_1[0, 0], r*q_1[1, 0], r*q_1[2, 0],
            q_1[0, 1], q_1[1, 1], q_1[2, 1],
            length=0.25, color="green"
        )
        writer.grab_frame()